In [17]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import os

In [18]:
os.chdir(r'C:\Users\aminm\OneDrive\Documents\Programming\Python\Program')

with open('negative.txt', 'r') as f:
    neg_words = f.read().split()
with open('positive.txt', 'r') as f:
    pos_words = f.read().split()
with open('negation.txt', 'r') as f:
    negation_words = f.read().split()
with open('booster_inc.txt', 'r') as f:
    multinc_words = f.read().split()
with open('booster_decr.txt', 'r') as f:
    multdec_words = f.read().split()

tweets = []
with open('hellooo.dat', 'r', encoding='utf-8', errors="ignore") as f:
    tweets = f.read().split('\n')
while("" in tweets) :
    tweets.remove("")

In [32]:
lemma = WordNetLemmatizer()
def text1_prep(x: str) -> list:
    corp = str(x).lower()
    corp = corp.strip('!"#$%&()*+,-./:;<=>?@[\\]^_{|}~')
    tokens = corp.split(' ')
    lemmatize = [lemma.lemmatize(w) for w in tokens]
    return lemmatize

stop_words = stopwords.words('english')
def text2_prep(x: str) -> list:
    corp = str(x).lower()
    corp = re.sub('[^a-zA-Z]+',' ', corp).strip()
    tokens = word_tokenize(corp)
    words = [t for t in tokens if t not in stop_words]
    lemmatize = [lemma.lemmatize(w) for w in words]
    return lemmatize

def get_negated(magnitude, negation_bool):
    negated = magnitude*-1 if negation_bool else magnitude
    return negated

def get_scalar1(negated, multi_low_bool):
    scalar1 = negated*0.5 if multi_low_bool else negated 
    return  scalar1 

def get_scalar2(scalar1, multi_high_bool):
    scalar2 = scalar1*1.5 if multi_high_bool else scalar1 
    return scalar2 

def get_sentiment(scalar2):
    sentiment = []
    for i in scalar2:
        if i>0:
            sentiment.append("Positive")
        elif i<0:
            sentiment.append("Negative")
        else:
            sentiment.append("Neutral")
    return sentiment
            
def get_overallScore(score):
    n = len(score)
    return round(sum(score)/n,2)

In [33]:
df = pd.DataFrame(tweets, columns= ['tweet'])
df['preprocess_txt1'] = df['tweet'].map(lambda x: text1_prep(x))
df['preprocess_txt2'] = df['tweet'].map(lambda x: text2_prep(x))
df['total_len'] = df['preprocess_txt2'].map(lambda x: len(x))
df['positive_count'] = df['preprocess_txt2'].map(lambda x: len([i for i in x if i in pos_words]))
df['negative_count'] = df['preprocess_txt2'].map(lambda x: len([i for i in x if i in neg_words]))
df['negation_bool'] = df['preprocess_txt1'].map(lambda x: any([i for i in x if i in negation_words]))
df['multi_low_bool'] = df['preprocess_txt1'].map(lambda x: any([i for i in x if i in multdec_words]))
df['multi_high_bool'] = df['preprocess_txt1'].map(lambda x: any([i for i in x if i in multinc_words]))
df['magnitude'] = df['positive_count']-df['negative_count']
df['negated'] = df.apply(lambda x: get_negated(x.magnitude, x.negation_bool), axis=1)
df['scalar1'] = df.apply(lambda x: get_scalar1(x.negated, x.multi_low_bool), axis=1)
df['scalar2'] = df.apply(lambda x: get_scalar2(x.scalar1, x.multi_high_bool), axis=1)
df['Sentiment'] = get_sentiment(df['scalar2'])
df['Sentiment Score'] = round(df['scalar2']/df['total_len'],2)

In [34]:
df

,tweet,preprocess_txt1,preprocess_txt2,total_len,positive_count,negative_count,negation_bool,multi_low_bool,multi_high_bool,magnitude,negated,scalar1,scalar2,Sentiment,Sentiment Score
0,01-18-2022 10:10 - Currently 16 stream gages i...,"[01-18-2022, 10:10, -, currently, 16, stream, ...","[currently, stream, gage, u, nw, floodstage, f...",8,0,0,False,False,False,0,0,0.0,0.0,Neutral,0.00
1,"@Lintang059 Hi, Im Disaster Bot. To report flo...","[lintang059, hi,, im, disaster, bot., to, repo...","[lintang, hi, im, disaster, bot, report, flood...",13,0,1,False,False,False,-1,-1,-1.0,-1.0,Negative,-0.08
2,Fist tweet #banjir #selasa #jktinfo,"[fist, tweet, #banjir, #selasa, #jktinfo]","[fist, tweet, banjir, selasa, jktinfo]",5,0,1,False,False,False,-1,-1,-1.0,-1.0,Negative,-0.20
3,RT @charlielangton: What an icy 🥶 mess on Harl...,"[rt, @charlielangton:, what, an, icy, 🥶, mess,...","[rt, charlielangton, icy, mess, harlow, street...",16,0,2,False,False,False,-2,-2,-2.0,-2.0,Negative,-0.12
4,RT @bigbadwolfbooks: RT this post and 1 brand ...,"[rt, @bigbadwolfbooks:, rt, this, post, and, 1...","[rt, bigbadwolfbooks, rt, post, brand, new, bo...",10,0,0,False,False,False,0,0,0.0,0.0,Neutral,0.00
5,Grab these books at further markdown prices! 😍…,"[grab, these, book, at, further, markdown, pri...","[grab, book, markdown, price]",4,0,0,False,False,False,0,0,0.0,0.0,Neutral,0.00
6,RT @animationii: A humanitarian disaster occur...,"[rt, @animationii:, a, humanitarian, disaster,...","[rt, animationii, humanitarian, disaster, occu...",15,0,1,False,False,False,-1,-1,-1.0,-1.0,Negative,-0.07
7,RT @mjsamberg: Do yourself a favor and listen ...,"[rt, @mjsamberg:, do, yourself, a, favor, and,...","[rt, mjsamberg, favor, listen, dr, dudley, flo...",15,1,0,False,False,False,1,1,1.0,1.0,Positive,0.07
8,“Do not take for granted the things closest to...,"[“do, not, take, for, granted, the, thing, clo...","[take, granted, thing, closest, heart, cling, ...",14,0,0,True,False,False,0,0,0.0,0.0,Neutral,0.00
9,Current Temp at Hamilton Str. Prkng Lot D on 0...,"[current, temp, at, hamilton, str., prkng, lot...","[current, temp, hamilton, str, prkng, lot, fee...",14,1,0,False,False,False,1,1,1.0,1.0,Positive,0.07


In [22]:
pd.set_option('display.max_rows', df.shape[0]+1)

In [23]:
df['Sentiment Score']
#print(df.loc[0])

0      0.00
1     -0.08
2     -0.20
3     -0.12
4      0.00
5      0.00
6     -0.07
7      0.07
8      0.00
9      0.07
10     0.00
11     0.00
12     0.00
13    -0.14
14     0.00
15     0.00
16    -0.20
17     0.00
18     0.00
19     0.00
20     0.00
21     0.00
22    -0.25
23     0.00
24     0.08
25     0.00
26     0.00
27     1.00
28     0.00
29     0.00
30     0.00
31     0.00
32     0.00
33     0.00
34     0.00
35     0.00
36     0.00
37     0.09
38     0.00
39     0.33
40     0.00
41     0.00
42     0.20
43     0.00
44     0.00
45     0.13
46    -0.19
47     0.00
48    -0.29
49     0.12
50    -0.08
51     0.25
52     0.00
53     0.20
54     0.00
55     0.00
56    -0.12
57     0.20
58     0.00
59     0.00
60    -0.17
61     0.00
62     0.33
63     0.00
64     0.14
65     0.20
66     0.00
67     0.00
68    -0.17
69     0.00
70     0.15
71     0.11
72     0.00
73     0.36
74     0.27
75     0.00
76     0.14
77     0.00
78     0.33
79     0.00
80     0.00
81    -0.12
82     0.00
83  

In [24]:
print(get_overallScore(df['Sentiment Score']))

0.03
